# Protein Classification Challenge – Fourth Challenge
 
**Name:** AJ Book  
**Course:** EN.605.656.8VL – Computational Drug Discovery and Development  
**Due Date:** 04/21/2025  
 


This module begins by loading the preprocessed training set (`metadata_org_w_features.csv`) and the held‐out evaluation set (`testing_data_w_features.csv`).  Feature augmentation is then performed by merging:

1. **Pfam** domain presence/absence profiles  
2. **Peptide descriptors** (Kidera factors, VHSE-8, Atchley factors, Z-scales)  
3. **Protlearn** physicochemical descriptors (e.g., Shannon entropy, autocorrelation)  
4. **ProtBert** contextual embeddings  
5. **ESM2** evolutionary language model embeddings  

Each feature block is validated for row and column consistency before proceeding to preprocessing, model training, evaluation, and final submission formatting.  


In [1]:
import re
import numpy as np
import pandas as pd
from collections import Counter
import itertools
from Bio.SeqUtils.ProtParam import ProteinAnalysis

from scipy.stats import randint, loguniform

from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.base import BaseEstimator
import random
from sklearn.model_selection import cross_validate
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
    confusion_matrix, classification_report
)

import matplotlib.pyplot as plt
import seaborn as sns

/home/secondbook5/micromamba/envs/comp_drug_disc/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


# Read in data

In [2]:
# Read in data
data = pd.read_csv("metadata_org_w_features.csv")
data_evaluation = pd.read_csv("testing_data_w_features.csv")

# Information and sample data

In [3]:
# information on data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 681 entries, 0 to 680
Columns: 589 entries, Entry to Gravy
dtypes: float64(584), int64(1), object(4)
memory usage: 3.1+ MB


In [4]:
# describe on data
data.describe()

,SequenceLength,A,C,D,E,F,G,H,I,K,...,DDD,MolecularWeight,IsoelectricPoint,Aromaticity,InstabilityIndex,Flexibility,Helix,Sheet,Turn,Gravy
count,681.000000,681.000000,681.000000,681.000000,681.000000,681.000000,681.000000,681.000000,681.000000,681.000000,...,681.000000,681.000000,681.000000,681.000000,681.000000,681.000000,681.000000,681.000000,681.000000,681.000000
mean,888.660793,0.071173,0.023348,0.044993,0.055845,0.046743,0.065586,0.022028,0.056255,0.047262,...,0.002206,99103.867857,7.097584,0.096863,42.160570,0.995021,0.304995,0.279123,0.389817,-0.051027
std,827.859592,0.022832,0.012300,0.013231,0.017925,0.015308,0.018840,0.007750,0.018204,0.017596,...,0.003108,92308.456460,1.551349,0.021673,7.412366,0.006859,0.029081,0.032611,0.043874,0.325872
min,103.000000,0.024283,0.000000,0.007605,0.005714,0.007782,0.016807,0.004065,0.009840,0.002915,...,0.000000,11710.270000,4.383673,0.033074,15.359896,0.974438,0.222539,0.183150,0.248630,-0.757339
25%,384.000000,0.056695,0.015251,0.037879,0.044280,0.035479,0.052764,0.016825,0.044171,0.034237,...,0.000000,42771.083500,5.723255,0.082587,37.259593,0.991538,0.284024,0.257261,0.361333,-0.284269
50%,676.000000,0.067460,0.021417,0.046046,0.058594,0.044747,0.065963,0.021544,0.056726,0.048017,...,0.001377,74697.663500,6.556855,0.094744,42.218182,0.996686,0.302326,0.277000,0.382857,-0.120350
75%,1015.000000,0.081784,0.030753,0.053254,0.067669,0.055838,0.078212,0.026769,0.068000,0.058943,...,0.003229,112207.097800,8.660054,0.109974,46.958534,0.999503,0.323615,0.300946,0.416667,0.136484
max,7096.000000,0.164000,0.074873,0.116598,0.123102,0.110701,0.153509,0.056093,0.125461,0.104956,...,0.031637,794048.925800,11.114366,0.180812,66.024733,1.013136,0.403005,0.419863,0.546125,1.008772


In [5]:
data.head()

,Entry,CleanSequence,Selected_PDB,ProteinClass,SequenceLength,A,C,D,E,F,...,DDD,MolecularWeight,IsoelectricPoint,Aromaticity,InstabilityIndex,Flexibility,Helix,Sheet,Turn,Gravy
0,P21611,MGKAAAVVLVTLVALLGLAQADLTPKVQVYSRFPASAGTKNVLNCF...,3p73,MHC,119,0.109244,0.016807,0.058824,0.042017,0.058824,...,0.000000,13041.7928,5.834204,0.109244,22.404202,0.996347,0.336134,0.277311,0.369748,-0.047059
1,Q66GT5,MAASAWLEAGLARVLFYPTLLYTVFRGRVRGPAHRDWYHRIDHTVL...,3rgo,Phosphatase,193,0.108808,0.015544,0.020725,0.067358,0.025907,...,0.000000,21942.2575,9.739448,0.082902,37.915078,0.992205,0.367876,0.186528,0.383420,-0.161140
2,Q9Y006,MNLTIKEEDFTNTFMKNEESFNTFRVTKVKRWNAKRLFKILFVTVF...,3qvc,Protease,451,0.028825,0.008869,0.048780,0.068736,0.079823,...,0.002227,51692.5641,8.043414,0.137472,37.618204,1.002078,0.305987,0.290466,0.434590,-0.249667
3,P05622,MGLPGVIPALVLRGQLLLSVLWLLGPQTSRGLVITPPGPEFVLNIS...,1aya,RTK,1098,0.043716,0.017304,0.056466,0.069217,0.030055,...,0.003650,122788.2434,4.994010,0.083789,47.564763,0.999045,0.281421,0.311475,0.385246,-0.203097
4,P06343,MALQIPSLLLLAAVVVLTVLSSPGTEGGNSERHFVHQFQPFCYFTN...,1d9k,MHC,263,0.041825,0.019011,0.022814,0.076046,0.038023,...,0.000000,29966.8989,8.991680,0.091255,46.750989,0.995690,0.262357,0.254753,0.399240,-0.303042


# Extract the relevant fields from the dataset

In [6]:
def extract_fields_from_data(data):
    column_start = data.columns.get_loc("SequenceLength")+1
    # Entry and ProteinClass
    df = data.loc[:,['Entry', 'ProteinClass']]
    # PDB
    selected_PDB = data.loc[:,['Selected_PDB']]
    # Sequence
    seq = data.loc[:,['CleanSequence']]
    # Sequence Length
    seq_L = data.loc[:,['SequenceLength']]
    # Amino Acid Frequencies
    freq = data.iloc[:, column_start : column_start+20]
    # All possible dipeptide frequencies
    dipep = data.iloc[:, column_start+20 : column_start+20 + 400]
    # Reduced Amino Acid Alphabet Frequencies
    red_freq = data.iloc[:, column_start+20 + 400 : column_start+20 + 400 + 5]
    # N-Gram Profiles of Reduced Amino Acid Alphabet
    red_ngram = data.iloc[:, column_start+20 + 400 + 5 : column_start+20 + 400 + 5 + 150]
    # Protein Properties
    prop = data.iloc[:, column_start+20 + 400 + 5 + 150 :]

    return df, selected_PDB, seq, seq_L, freq, dipep, red_freq, red_ngram, prop

# For data with no ProteinClass
def extract_fields_from_data_evaluation(data):
    column_start = data.columns.get_loc("SequenceLength")+1
    # Entry and ProteinClass
    df = data.loc[:,['Entry']]
    # PDB
    selected_PDB = data.loc[:,['Selected_PDB']]
    # Sequence
    seq = data.loc[:,['CleanSequence']]
    # Sequence Length
    seq_L = data.loc[:,['SequenceLength']]
    # Amino Acid Frequencies
    freq = data.iloc[:, column_start : column_start+20]
    # All possible dipeptide frequencies
    dipep = data.iloc[:, column_start+20 : column_start+20 + 400]
    # Reduced Amino Acid Alphabet Frequencies
    red_freq = data.iloc[:, column_start+20 + 400 : column_start+20 + 400 + 5]
    # N-Gram Profiles of Reduced Amino Acid Alphabet
    red_ngram = data.iloc[:, column_start+20 + 400 + 5 : column_start+20 + 400 + 5 + 150]
    # Protein Properties
    prop = data.iloc[:, column_start+20 + 400 + 5 + 150 :]

    return df, selected_PDB, seq, seq_L, freq, dipep, red_freq, red_ngram, prop

In [7]:
(df, selected_PDB, seq, seq_L, freq, dipep,
 red_freq, red_ngram, prop) = extract_fields_from_data(data)

# Evaluation data
(df_evaluation, selected_PDB_evaluation, seq_evaluation, seq_L_evaluation,
  freq_evaluation, dipep_evaluation, red_freq_evaluation, red_ngram_evaluation, prop_evaluation) = extract_fields_from_data_evaluation(data_evaluation)

# You can use this section if you wanted to add more features to the data

## Merge External Features into Dataset
To enhance protein classification, external biological and structural features are incorporated. These include Pfam domain annotations, peptide physicochemical descriptors, intrinsic disorder predictions (AIUPred), and deep embeddings (ProtBert, ESM2).

In [8]:
# Feel free to add other feature to this data

# --- Step 1: Merge extracted Base features ---
# You already extracted: df, freq, dipep, red_freq, red_ngram, prop

# Merge base feature blocks into one
base_features = pd.concat([freq, dipep, red_freq, red_ngram, prop], axis=1)

# Combine base features with Entry and ProteinClass
df_base = pd.concat([df, base_features], axis=1)

print(f"Base features merged shape: {df_base.shape}")
print("Sample columns:", df_base.columns[:10].tolist())


# --- Step 2: Prefix Base Features with 'base_' ---

# Identify all columns that are actual features (not Entry or ProteinClass)
base_feature_cols = [col for col in df_base.columns if col not in ["Entry", "ProteinClass"]]

# Rename these columns by prefixing with 'base_'
df_base.rename(columns={col: f"base_{col}" for col in base_feature_cols}, inplace=True)

print(f"Prefixed base feature columns. Now total columns: {len(df_base.columns)}")
print(f" Sample prefixed columns: {df_base.columns[:10].tolist()}")



Base features merged shape: (681, 586)
Sample columns: ['Entry', 'ProteinClass', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'I']
Prefixed base feature columns. Now total columns: 586
 Sample prefixed columns: ['Entry', 'ProteinClass', 'base_A', 'base_C', 'base_D', 'base_E', 'base_F', 'base_G', 'base_H', 'base_I']


In [9]:
# --- Step 2: Load ProtLearn and Peptide descriptors ---

protlearn_df = pd.read_csv("protlearn_features.csv")
peptide_df = pd.read_csv("peptide_descriptors.csv")

# Clean column names
protlearn_df.columns = protlearn_df.columns.str.strip()
peptide_df.columns = peptide_df.columns.str.strip()

# Insert Entry column manually
protlearn_df.insert(0, "Entry", data["Entry"].values)  # assumes correct order
peptide_df.insert(0, "Entry", data["Entry"].values)

print("ProtLearn columns:", protlearn_df.columns[:5])
print("Peptide columns:", peptide_df.columns[:5])



# Start from existing correctly prefixed df_base
# Merge ProtLearn and Peptide descriptors

df_full = df_base.merge(protlearn_df, on="Entry", how="left")
df_full = df_full.merge(peptide_df, on="Entry", how="left")

print(f"Full merged feature matrix shape after adding ProtLearn and Peptides: {df_full.shape}")

# Build modeling matrices
X_full = df_full.drop(columns=["Entry", "ProteinClass"])  # only numeric features
y_full = df_full[["Entry", "ProteinClass"]]                # Entry and class labels

print("X_full shape (features only):", X_full.shape)
print("y_full shape (Entry + Label):", y_full.shape)


ProtLearn columns: Index(['Entry', 'PL_AAIndex1_0', 'PL_AAIndex1_1', 'PL_AAIndex1_2',
       'PL_AAIndex1_3'],
      dtype='object')
Peptide columns: Index(['Entry', 'PEP_AF1', 'PEP_AF2', 'PEP_AF3', 'PEP_AF4'], dtype='object')
Full merged feature matrix shape after adding ProtLearn and Peptides: (681, 2392)
X_full shape (features only): (681, 2390)
y_full shape (Entry + Label): (681, 2)


In [10]:
df_full.head()

,Entry,ProteinClass,base_A,base_C,base_D,base_E,base_F,base_G,base_H,base_I,...,PEP_VSTPV2,PEP_VSTPV3,PEP_VSTPV4,PEP_VSTPV5,PEP_VSTPV6,PEP_Z1,PEP_Z2,PEP_Z3,PEP_Z4,PEP_Z5
0,P21611,MHC,0.109244,0.016807,0.058824,0.042017,0.058824,0.058824,0.025210,0.016807,...,0.027647,0.413613,-0.383109,0.097479,-0.350420,-0.116134,-0.569580,-0.063361,-0.327899,0.277479
1,Q66GT5,Phosphatase,0.108808,0.015544,0.020725,0.067358,0.025907,0.051813,0.041451,0.046632,...,0.150000,0.398601,-0.355959,0.152902,-0.195492,0.006114,-0.442176,-0.514404,-0.167306,0.208187
2,Q9Y006,Protease,0.028825,0.008869,0.048780,0.068736,0.079823,0.053215,0.015521,0.062084,...,0.104235,0.298448,-0.326984,0.184612,-0.244235,-0.061197,-0.288692,-0.267118,-0.491641,0.207406
3,P05622,RTK,0.043716,0.017304,0.056466,0.069217,0.030055,0.061020,0.021858,0.050091,...,0.063643,0.376421,-0.366494,0.159189,-0.229271,0.004599,-0.493270,-0.203634,-0.507723,0.244117
4,P06343,MHC,0.041825,0.019011,0.022814,0.076046,0.038023,0.072243,0.030418,0.041825,...,0.108479,0.340684,-0.393688,0.182548,-0.224068,0.076274,-0.435741,-0.441939,-0.360951,0.158023


In [11]:
import pandas as pd
from pathlib import Path

# Step 1: Define clean function to load Pfam hits

def load_clean_pfam(domtblout_path: Path, entries: pd.Series, e_cut: float = 1e-3) -> pd.DataFrame:
    """
    Parse hmmscan domtblout.txt and build a clean binary Pfam domain hit matrix.

    Args:
        domtblout_path (Path): Path to domtblout.txt file.
        entries (pd.Series): List of Entry IDs (protein IDs).
        e_cut (float): E-value cutoff for accepting a domain hit.

    Returns:
        pd.DataFrame: Binary matrix (Entry × PfamAcc), 1 = domain present, 0 = absent.
    """
    rows = []
    with domtblout_path.open() as fh:
        for line in fh:
            if line.startswith("#"):
                continue
            parts = line.strip().split()
            if len(parts) < 23:
                continue
            pfam_acc = parts[1]
            entry = parts[3]
            evalue = float(parts[6])
            if evalue <= e_cut:
                rows.append((entry, pfam_acc))

    hits_df = pd.DataFrame(rows, columns=["Entry", "PfamAcc"])
    pfam_matrix = pd.crosstab(hits_df["Entry"], hits_df["PfamAcc"])
    pfam_matrix = pfam_matrix.reindex(entries.values, fill_value=0)
    pfam_matrix.index.name = "Entry"

    return pfam_matrix

# Step 2: Load clean Pfam domains

domtblout_path = Path("data/pfam/domtblout.txt")

pfam_df_clean = load_clean_pfam(domtblout_path, df_full["Entry"])
pfam_df_clean = pfam_df_clean.reset_index()

print(f"Clean Pfam matrix shape: {pfam_df_clean.shape}")
print(f"Example Pfam columns: {pfam_df_clean.columns[1:6].tolist()}")

# Step 3: Check PF-prefixed columns BEFORE dropping

# Find PF-prefixed columns
pfam_cols_in_full = [c for c in df_full.columns if c.startswith("PF")]

print(f"Found {len(pfam_cols_in_full)} columns starting with 'PF':")
print(pfam_cols_in_full)

# Optional: Preview a few values
print("\nPreview of existing PF-prefixed columns before dropping:")
print(df_full[["Entry"] + pfam_cols_in_full].head())




Clean Pfam matrix shape: (681, 529)
Example Pfam columns: ['PF00001.24', 'PF00002.27', 'PF00003.25', 'PF00004.32', 'PF00005.30']
Found 0 columns starting with 'PF':
[]

Preview of existing PF-prefixed columns before dropping:
    Entry
0  P21611
1  Q66GT5
2  Q9Y006
3  P05622
4  P06343


In [12]:
# (Pause here to manually review if needed)

# Step 4: Drop old PF columns after verifying

# Only proceed after verifying!
df_full = df_full.drop(columns=pfam_cols_in_full, errors="ignore")
print(f"Dropped {len(pfam_cols_in_full)} old PF-prefixed columns before clean merge.")

# Step 5: Merge clean Pfam matrix into df_full

df_full = df_full.merge(pfam_df_clean, on="Entry", how="left").fillna(0)
print(f"Final df_full shape after clean Pfam merge: {df_full.shape}")

# Step 6: Final Sanity Check

pfam_columns = [c for c in df_full.columns if c.startswith("PF")]
print(f"Final number of Pfam domain features: {len(pfam_columns)}")
print("\nPreview of final clean Pfam features:")
print(df_full[["Entry"] + pfam_columns[:5]].head(10))

Dropped 0 old PF-prefixed columns before clean merge.
Final df_full shape after clean Pfam merge: (681, 2920)
Final number of Pfam domain features: 528

Preview of final clean Pfam features:
    Entry  PF00001.24  PF00002.27  PF00003.25  PF00004.32  PF00005.30
0  P21611           0           0           0           0           0
1  Q66GT5           0           0           0           0           0
2  Q9Y006           0           0           0           0           0
3  P05622           0           0           0           0           0
4  P06343           0           0           0           0           0
5  P50281           0           0           0           0           0
6  Q13255           0           0           1           0           0
7  P08069           0           0           0           0           0
8  B0V2N1           0           0           0           0           0
9  Q9Y5Z0           0           0           0           0           0


In [13]:
# --- See what Pfam columns look like after merging ---

# Step 1: Find all Pfam domain columns
pfam_columns = [c for c in df_full.columns if c.startswith("PF")]

print(f" Found {len(pfam_columns)} Pfam domain features.")

# Step 2: Look at the first few Pfam columns and first few proteins
print(df_full[["Entry"] + pfam_columns[:5]].head(10))

# Step 3: Look for proteins that have no Pfam hits
no_pfam = df_full[pfam_columns].sum(axis=1) == 0

print(f" Proteins with NO Pfam hits: {no_pfam.sum()} out of {len(df_full)}")

# See a few proteins with no Pfam hits
print(df_full.loc[no_pfam, ["Entry"] + pfam_columns[:5]].head())


 Found 528 Pfam domain features.
    Entry  PF00001.24  PF00002.27  PF00003.25  PF00004.32  PF00005.30
0  P21611           0           0           0           0           0
1  Q66GT5           0           0           0           0           0
2  Q9Y006           0           0           0           0           0
3  P05622           0           0           0           0           0
4  P06343           0           0           0           0           0
5  P50281           0           0           0           0           0
6  Q13255           0           0           1           0           0
7  P08069           0           0           0           0           0
8  B0V2N1           0           0           0           0           0
9  Q9Y5Z0           0           0           0           0           0
 Proteins with NO Pfam hits: 0 out of 681
Empty DataFrame
Columns: [Entry, PF00001.24, PF00002.27, PF00003.25, PF00004.32, PF00005.30]
Index: []


In [14]:
import numpy as np

# ---  Load ProtBERT embeddings (training set) ---

protbert_path = "embed_models/protbert_train_full.npy"
protbert_embeddings = np.load(protbert_path)  # Should be shape (681, 1024)

# Convert to DataFrame and add Entry
protbert_df = pd.DataFrame(
    protbert_embeddings,
    columns=[f"ProtBERT_{i}" for i in range(protbert_embeddings.shape[1])]
)
protbert_df.insert(0, "Entry", df_full["Entry"].values)

print(f"ProtBERT embedding DataFrame shape: {protbert_df.shape}")
print(f"Sample ProtBERT columns: {protbert_df.columns[:5].tolist()}")

# ---  Merge ProtBERT embeddings ---

df_full = df_full.merge(protbert_df, on="Entry", how="left")
print(f"Shape after merging ProtBERT embeddings: {df_full.shape}")

# ---  Load ESM2 embeddings (training set) ---

esm2_path = "embed_models/esm2_train_full.npy"
esm2_embeddings = np.load(esm2_path)  # Should be shape (681, 1280)

# Convert to DataFrame and add Entry
esm2_df = pd.DataFrame(
    esm2_embeddings,
    columns=[f"ESM2_{i}" for i in range(esm2_embeddings.shape[1])]
)
esm2_df.insert(0, "Entry", df_full["Entry"].values)

print(f"ESM2 embedding DataFrame shape: {esm2_df.shape}")
print(f"Sample ESM2 columns: {esm2_df.columns[:5].tolist()}")

# ---  Merge ESM2 embeddings ---

df_full = df_full.merge(esm2_df, on="Entry", how="left")
print(f"Shape after merging ESM2 embeddings: {df_full.shape}")

# ---  Final Sanity Check ---

print("\nFinal columns after embedding merge:")
print(df_full.columns[:10])

# Build final modeling matrices
X_full = df_full.drop(columns=["Entry", "ProteinClass"])  # Only features
y_full = df_full[["Entry", "ProteinClass"]]                # Entry + Class label

print(f" Final X_full shape: {X_full.shape}")
print(f" Final y_full shape: {y_full.shape}")


ProtBERT embedding DataFrame shape: (681, 1025)
Sample ProtBERT columns: ['Entry', 'ProtBERT_0', 'ProtBERT_1', 'ProtBERT_2', 'ProtBERT_3']
Shape after merging ProtBERT embeddings: (681, 3944)
ESM2 embedding DataFrame shape: (681, 1281)
Sample ESM2 columns: ['Entry', 'ESM2_0', 'ESM2_1', 'ESM2_2', 'ESM2_3']
Shape after merging ESM2 embeddings: (681, 5224)

Final columns after embedding merge:
Index(['Entry', 'ProteinClass', 'base_A', 'base_C', 'base_D', 'base_E',
       'base_F', 'base_G', 'base_H', 'base_I'],
      dtype='object')
 Final X_full shape: (681, 5222)
 Final y_full shape: (681, 2)


In [15]:
# --- Step 5: Final Data Double-Check ---

# Check for missing values
n_missing_X = X_full.isnull().sum().sum()
n_missing_y = y_full.isnull().sum().sum()

print(f" Missing values in X_full: {n_missing_X}")
print(f"Missing values in y_full: {n_missing_y}")

assert n_missing_X == 0, " Found missing values in X_full after merging!"
assert n_missing_y == 0, " Found missing values in y_full after merging!"

print(" Double-check passed: No missing values.")

 Missing values in X_full: 0
Missing values in y_full: 0
 Double-check passed: No missing values.


In [16]:
df_full.head()

,Entry,ProteinClass,base_A,base_C,base_D,base_E,base_F,base_G,base_H,base_I,...,ESM2_1270,ESM2_1271,ESM2_1272,ESM2_1273,ESM2_1274,ESM2_1275,ESM2_1276,ESM2_1277,ESM2_1278,ESM2_1279
0,P21611,MHC,0.109244,0.016807,0.058824,0.042017,0.058824,0.058824,0.025210,0.016807,...,0.004602,-0.082712,-0.138449,-0.015294,-0.006374,-0.047119,0.072656,-0.077782,-0.014603,0.038721
1,Q66GT5,Phosphatase,0.108808,0.015544,0.020725,0.067358,0.025907,0.051813,0.041451,0.046632,...,-0.061771,-0.063909,-0.092090,-0.009350,0.042447,0.023802,0.101591,-0.167533,0.036926,0.024987
2,Q9Y006,Protease,0.028825,0.008869,0.048780,0.068736,0.079823,0.053215,0.015521,0.062084,...,0.183224,0.005093,-0.064172,0.079590,-0.049639,0.000521,0.145360,-0.155799,-0.135549,-0.052576
3,P05622,RTK,0.043716,0.017304,0.056466,0.069217,0.030055,0.061020,0.021858,0.050091,...,0.041743,0.005760,-0.123973,0.010424,0.021136,-0.079837,0.051444,-0.076187,0.032540,0.041431
4,P06343,MHC,0.041825,0.019011,0.022814,0.076046,0.038023,0.072243,0.030418,0.041825,...,0.032086,-0.010952,-0.108737,-0.008614,0.030564,0.006659,0.102912,-0.017738,0.003438,0.002770


# Construct Dataset with any of the extracted relevant fields

In [17]:
# This is an example of construction of the data with freq and red_freq, but you can add and make any field
Dataset = pd.concat([df, freq, red_freq, prop], axis=1)
Dataset_evaluation = pd.concat([df_evaluation, freq_evaluation, red_freq_evaluation, prop_evaluation], axis=1)

In [18]:
# Training Data with ProteinClass
Dataset.head()

,Entry,ProteinClass,A,C,D,E,F,G,H,I,...,D.1,MolecularWeight,IsoelectricPoint,Aromaticity,InstabilityIndex,Flexibility,Helix,Sheet,Turn,Gravy
0,P21611,MHC,0.109244,0.016807,0.058824,0.042017,0.058824,0.058824,0.025210,0.016807,...,0.100840,13041.7928,5.834204,0.109244,22.404202,0.996347,0.336134,0.277311,0.369748,-0.047059
1,Q66GT5,Phosphatase,0.108808,0.015544,0.020725,0.067358,0.025907,0.051813,0.041451,0.046632,...,0.088083,21942.2575,9.739448,0.082902,37.915078,0.992205,0.367876,0.186528,0.383420,-0.161140
2,Q9Y006,Protease,0.028825,0.008869,0.048780,0.068736,0.079823,0.053215,0.015521,0.062084,...,0.117517,51692.5641,8.043414,0.137472,37.618204,1.002078,0.305987,0.290466,0.434590,-0.249667
3,P05622,RTK,0.043716,0.017304,0.056466,0.069217,0.030055,0.061020,0.021858,0.050091,...,0.125683,122788.2434,4.994010,0.083789,47.564763,0.999045,0.281421,0.311475,0.385246,-0.203097
4,P06343,MHC,0.041825,0.019011,0.022814,0.076046,0.038023,0.072243,0.030418,0.041825,...,0.098859,29966.8989,8.991680,0.091255,46.750989,0.995690,0.262357,0.254753,0.399240,-0.303042


In [19]:
# Evaluation Data without ProteinClass
Dataset_evaluation.head()

,Entry,A,C,D,E,F,G,H,I,K,...,D.1,MolecularWeight,IsoelectricPoint,Aromaticity,InstabilityIndex,Flexibility,Helix,Sheet,Turn,Gravy
0,Q9LF79,0.081006,0.010242,0.054935,0.053073,0.031657,0.089385,0.016760,0.069832,0.065177,...,0.108007,116172.6688,7.865610,0.059590,33.680642,0.999017,0.323091,0.292365,0.384544,0.028026
1,P9WI81,0.108626,0.003195,0.073482,0.043131,0.023962,0.081470,0.019169,0.052716,0.028754,...,0.116613,66509.0117,5.223413,0.049521,28.408163,1.001449,0.263578,0.332268,0.349840,-0.173163
2,P04439,0.098630,0.013699,0.065753,0.063014,0.021918,0.079452,0.024658,0.035616,0.030137,...,0.128767,40840.2477,5.655275,0.090411,36.957014,1.000302,0.293151,0.265753,0.345205,-0.498630
3,Q16581,0.056017,0.031120,0.045643,0.031120,0.070539,0.045643,0.020747,0.051867,0.026971,...,0.076763,53863.6435,6.202180,0.114108,40.862697,0.989447,0.255187,0.307054,0.425311,0.204979
4,Q6QNK2,0.072082,0.020595,0.026316,0.040046,0.050343,0.064073,0.041190,0.050343,0.038902,...,0.066362,96528.7218,8.021753,0.110984,33.635481,0.990173,0.283753,0.288330,0.416476,0.155950


In [ ]:
## To Do ##
# Construct your own dataset with these field and any other field that you see fit
# Make sure you do both the training and evaluation data as haveing the same fields

# Processing Steps

In [ ]:
def processing_data(df):
    # Define X and y
    y = df['ProteinClass']
    X = df.drop(['Entry', 'ProteinClass'], axis=1)
    if 0:
        print("Feature sample:")
        print(X.head())
        print("\nTarget sample:")
        print(y.head())

    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42)

    # Feature scaling (important for logistic regression, gradient boosting, MLP, etc.)
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)


    # Defined labels
    labels = np.array(['ATPase', 'Aquaporin', 'Channel', 'GPCR', 'Integrin', 'MHC',
       'Phosphatase', 'Protease', 'RTK', 'Ser:Thr'])
    # Create a Label Encoder
    label_encoder = LabelEncoder()
    # Fit the label encoder on all possible labels, will make the labels [0, 1, 2, 3 ...]
    label_encoder.fit(labels)
    # Transform both training and testing labels
    y_label_encoded = label_encoder.transform(y)
    y_train_label_encoded = label_encoder.transform(y_train)
    y_test_label_encoded = label_encoder.transform(y_test)
    # print('first sample label encoded:', y_train_label_encoded[0])

    return (label_encoder, X.values, X_scaled, X_train, X_train_scaled, X_test, X_test_scaled,
            y, y_label_encoded, y_train, y_train_label_encoded, y_test, y_test_label_encoded, scaler)


def processing_data_evaluation(df,scaler):
    # Define X
    X = df.drop(['Entry'], axis=1)

    # Feature scaling (important for logistic regression, gradient boosting, MLP, etc.)
    X_scaled = scaler.fit_transform(X)

    return (X.values, X_scaled)


In [ ]:
(label_encoder, X, X_scaled, X_train, X_train_scaled, X_test, X_test_scaled, y, y_label_encoded,
y_train, y_train_label_encoded, y_test, y_test_label_encoded, scaler)  = processing_data(Dataset)

# Evaluation data
X_evaluation, X_scaled_evaluation = processing_data_evaluation(Dataset_evaluation, scaler)

# Creating Model

In [ ]:
# random model
class random_model:
    def __init__(self):
        pass
    def fit(self,X, y):
        pass
    def predict(self,X):
        pred = []
        for x in X:
            pred.append(np.random.randint(10))
        return np.array(pred)
    def get_params(self, deep=True):
        return {}

# Initialize my random model
my_model = random_model()
# Initialize Logistic Regression model
lr_model = LogisticRegression()

## To Do ##
# Create 2 more models here like random forest, leanear model, etc and initialize it
# you can use sklear, torch, etc

# Train Model

In [ ]:
# Fit My Model to the training data
my_model.fit(X_train, y_train)

# Fit the model to the training data without tuning
lr_model.fit(X_train_scaled, y_train)

## To Do ##
# Fit your models to the training data you can use
# raw, scaled, or other forms

# Training and Tuning

In [ ]:
# Define the parameter distribution for 'C'
param_dist_lr = {
    'C': loguniform(1e-3, 1e3),
    'penalty': ['l2'],
    'solver': ['lbfgs'],
    'multi_class': ['multinomial'],
    'max_iter': [500, 1000, 2000],
    'random_state': [42]
}

# Set up RandomizedSearchCV
random_search_lr = RandomizedSearchCV(
    estimator=lr_model,
    param_distributions=param_dist_lr,
    n_iter=50,
    cv=5,
    scoring='accuracy',
    random_state=42,
    n_jobs=-1
)

# Fit the model to the training data with tuning
random_search_lr.fit(X_train_scaled, y_train)
# Get the best estimator
best_lr_model = random_search_lr.best_estimator_
print("Best parameters found for Logistic Regression:")
print(random_search_lr.best_params_)

## To Do ##
# Fit your models to the training data you can use
# raw, scaled, or other forms
# Make sure to do tuning this time

# Evaluating model

In [ ]:
def evaluate_model(model, X_test, y_test, model_name):

    y_pred = model.predict(X_test)

    # Evaluation
    accuracy = accuracy_score(y_test, y_pred)
    print(f"\n=== {model_name} ===")
    print(f"Accuracy for testing data: {accuracy:.4f}")
    return y_pred

def evaluate_model_testing(model, X_test, model_name):

    y_pred = model.predict(X_test)

    print(f"\n=== {model_name} ===")
    print("Output Evaluated")
    return y_pred

In [ ]:
# Evaluate my model
y_pred_my_model = evaluate_model(my_model, X_test_scaled, y_test_label_encoded, model_name="My Random Model")
# Convert back to labels
y_pred_my_model = label_encoder.inverse_transform(y_pred_my_model)

# Evaluate Logistic Regression
y_pred_lr_model = evaluate_model(lr_model, X_test_scaled, y_test, model_name="Logistic Regression")


# Evaluate Logistic Regression Best estimator
y_pred_best_lr_model = evaluate_model(best_lr_model, X_test_scaled, y_test, model_name="Logistic Regression Best")



## To Do ##
# Evaluate all models


## Evaluate the provided data for the competition

In [ ]:
# Evaluate my model
y_pred_my_model = evaluate_model_testing(my_model, X_evaluation, model_name="My Random Model")
# Convert back to labels
y_pred_my_model_evaluation = label_encoder.inverse_transform(y_pred_my_model)


# Evaluate Logistic Regression
y_pred_lr_model_evaluation = evaluate_model_testing(lr_model, X_scaled_evaluation, model_name="Logistic Regression")


# Evaluate Logistic Regression Best Model
y_pred_lr_best_model_evaluation = evaluate_model_testing(best_lr_model, X_scaled_evaluation, model_name="Logistic Regression Best")

## To Do ##
# Evaluate all the models with the competition data


In [ ]:
# Prediction of my model on testing data
print("\n=== My Model ===")
print(y_pred_my_model_evaluation[:5])

# Prediction of my LR on testing data
print("\n=== Logistic Regression ===")
print(y_pred_lr_model_evaluation[:5])

# Prediction of my LR on testing data
print("\n=== Logistic Regression Best ===")
print(y_pred_lr_best_model_evaluation[:5])

## To Do ##
# Print the first 5 examples for all model output for the competition data
# Make sure that the output is converted back into a list of string for the Protien class

# Creat csv file for Evaluation

In [ ]:
# Function to creat csv output for uploading
def save_predictions(_fn, _y_pred, _df):
    import csv
    with open(_fn, 'w') as fout:
        writer = csv.writer(fout, delimiter=',', lineterminator='\n')
        writer.writerow(['Entry', 'ProteinClass'])
        for y, Entry in zip(_df['Entry'], _y_pred):
            writer.writerow([y, Entry])

In [ ]:
# Saving My random model output
save_predictions('Student_name_attempt_1.csv', y_pred_my_model_evaluation, Dataset_evaluation )
# Saving LR model output
save_predictions('Student_name_attempt_2.csv', y_pred_lr_model_evaluation, Dataset_evaluation )
# Saving Best LR model output
save_predictions('Student_name_attempt_3.csv', y_pred_lr_best_model_evaluation, Dataset_evaluation )

## To Do ##
# Save the output of your models

In [ ]:
## To Do ##
# Make sure to upload at least 3 attemps to the website
# Good Luck